In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
breast = pd.read_csv("breast-cancer.csv")

In [3]:
breast.value_counts()

Class                 age    menopause  tumor-size  inv-nodes  node-caps  deg-malig  breast  breast-quad  irradiat
no-recurrence-events  60-69  ge40       15-19       0-2        no         2          right   left_up      no          2
                      40-49  premeno    10-14       0-2        no         1          right   left_up      no          2
                      60-69  ge40       20-24       0-2        no         1          left    left_low     no          2
                      50-59  premeno    25-29       0-2        no         2          left    left_low     no          2
                      40-49  premeno    20-24       0-2        no         2          right   left_up      no          2
                                                                                                                     ..
                      50-59  ge40       20-24       3-5        yes        2          right   left_up      no          1
                                        25-29

In [ ]:
breast_cat = breast

In [8]:
le = preprocessing.LabelEncoder()
le.fit(breast['Class'])
le.transform(breast['Class'])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [18]:
U = breast['Class'].values.tolist()

freq_list = []

for c in set(U):
    freq_list.append(U.count(c)/len(U))

In [28]:
freq_list

[0.2972027972027972, 0.7027972027972028]

In [36]:
label = 'Class'

def I(U):
    U = U.values.tolist()
    freq_list = []
    I = 0

    for c in set(U):
        freq_list.append(U.count(c)/len(U))

    for i in range(len(freq_list)):
        I -= freq_list[i]*np.log(freq_list[i])
    return I

def Inf(d,U):
    inf = 0
    for i in range(len(U[d].unique())):
        inf += U[d].value_counts().values[i]/len(U) * I(U[label][U[d] == U[d].unique()[i]])
    return inf

def InfGain(d, U):
    return I(U[label]) - Inf(d,U)

In [37]:
print(I(breast['Class']))
print(Inf('menopause', breast))
print(InfGain('menopause', breast))

0.6084755754298137
0.6070881616542189
0.0013874137755948146


In [67]:
breast.columns

Index(['Class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps',
       'deg-malig', 'breast', 'breast-quad', 'irradiat'],
      dtype='object')

In [73]:
breast[breast.columns[6]].unique()

array([3, 2, 1], dtype=int64)

In [84]:
breast[breast.columns[6]].value_counts().values[0]

130

In [86]:
InfGain(breast.columns[6], breast)

-0.14913332788660627

In [34]:
for col in breast.columns:
    print(col, InfGain(col, breast))

Class 0.6084755754298137
age -0.37283291398549945
menopause -0.34066999877528903
tumor-size -0.5577137847987022
inv-nodes -2.2912443454471934
node-caps -0.856964397973043
deg-malig -0.14913332788660627
breast -0.034898986921418906
breast-quad -0.5171081649712768
irradiat -0.34359804648183034


In [13]:
from typing import Any, Iterable, Optional, Union
from math import log

def _entropy(y: pd.Series):
    class_counts = y.value_counts()
    total = class_counts.sum()

    def partial(count: int):
        frequency = count / total
        return -frequency * log(frequency)
    return class_counts.apply(partial).sum()


def _division_entropy(X: pd.DataFrame, y: pd.Series, division_label: Any):
    div_attr_counts = X[division_label].value_counts()
    total = div_attr_counts.sum()

    def partial(attr_value: int):
        count = div_attr_counts[attr_value]
        mask = X[division_label] == attr_value
        return count / total * _entropy(y[mask])
    return sum(div_attr_counts.index.map(partial))


def _division_info_gain(X: pd.DataFrame, y: pd.Series, division_label: Any):
    return _entropy(y) - \
        _division_entropy(X, y, division_label)


In [39]:
for col in breast.columns:
    print(_entropy(breast['Class']))
    print(_division_entropy(breast, breast['Class'], col))
    print(_division_info_gain(breast, breast['Class'], col))

0.6084755754298137
0.0
0.6084755754298137
0.6084755754298137
0.6011240865600114
0.007351488869802303
0.6084755754298137
0.6070881616542189
0.0013874137755948146
0.6084755754298137
0.5688475711018378
0.03962800432797586
0.6084755754298137
0.5606518246478241
0.04782375078198953
0.6084755754298137
0.5714458215749589
0.03702975385485474
0.6084755754298137
0.5550964132825851
0.0533791621472286
0.6084755754298137
0.6067503401148494
0.0017252353149642685
0.6084755754298137
0.5980321888324173
0.010443386597396409
0.6084755754298137
0.5905791918023827
0.017896383627430956


In [40]:
for col in breast.columns:
    print(I(breast['Class']))
    print(Inf(col, breast))
    print(InfGain(col, breast))

0.6084755754298137
0.0
0.6084755754298137
0.6084755754298137
0.6232991172547504
-0.014823541824936726
0.6084755754298137
0.6070881616542189
0.0013874137755948146
0.6084755754298137
0.5602642232380364
0.0482113521917773
0.6084755754298137
0.5592594917141993
0.04921608371561437
0.6084755754298137
0.5714458215749589
0.03702975385485474
0.6084755754298137
0.5819094959800177
0.026566079449795943
0.6084755754298137
0.6067503401148494
0.0017252353149642685
0.6084755754298137
0.6179878333327674
-0.009512257902953736
0.6084755754298137
0.5905791918023827
0.017896383627430956
